## Opis testownych modelów
W projekcie skupiam się na dobieraniu odpowiedniej konstrukcji sieci konwolucyjnej. Model ten został wybrany, ponieważ jego założenia idealnie pasują do postawionego problemu - nauczenia komputera odpowiadania na dane CIFAR10.

Sieci konwolucyjne skupiają się na odnajdywaniu charakterystych cech danych wejściowych. Zakładają one że dane wejściowe są obrazami, niekoniecznie dwuwymiarowymi. Pierwsze ukryte wartwy często uczą się wykrywać krawędzie na podstawie samych pikseli, podczas gdy głębsze warstwy są już w stanie wykrywać kształty. 

Sieci konwolucyjne generalnie budowane są z warstw:
- konwolucyjnych - dopasowujących filtr o danym rozmiarze do fragmentu obrazka. mnożone są wagi filtru przez poszczególne piksele
- aktywacji - pozwalających na dodawanie nieliniowych zależności. Często używane jest ReLU które odrzuca wartości ujemne, pozostawiając dodanie
- zmniejszania wymiaru - tzw. pooling. Mają na celu redukcje ilości parametrów oraz obliczeń, pomagając unikać overfitowania 
- gęstych - znanych z klasycznych sieci. uczą się odpowiadać na podstawie danych z konwolucji. 

## Dane
Wejście to obrazek RGB 32x32x3. Wyjście to to liczba naturalna z przedziału [0,9]. Dane pochodą z datasetu CIFAR10. Kazdy obrazek przedstawia jedną klasę z: samolotów, samochodów, ptaków, kotów, sarenek, psów, żab, koni, statów i ciężarówek.  										
 
Dla celów nauczania wyjście zostało przetransformowane to 10 wymiarowej przestrzeni, tak, że 1->[1,0,...,0], 5->[0,0,0,0,0,1,0,0,0,0], 9->[...,0,1]. Motywacją jest to, żeby sieć nie uważała, że samochód o klasie 1, może być czymś pomiędzy samolotem(0) a ptakiem(2).

## Hiperparametry

Na poniższych parametrach przeprowadzony został random search.
- optimizer: 
    - adam: 
        - lr z przedziału (0.01, 0.1)
        - beta_1, beta_2 z przedziału (0.9, 0.999)
    - adadelta: 
        - lr z przedziału (0.01, 0.1)
        - rho z przedziału (0.01, 0.99)
    - adagrad: 
        - lr z przedziału (0.01, 0.1)
- loss:
    - categorical_crossentropy
    - mean_squared_error
    - mean_squared_error
- warstwy gęste, każda o rozmairze z [32, 64, 128]
    - 3
    - 2
    - 1
- warstwy konwolucji, każda o rozmairze z [32, 64, 128] i kernel z [(2,2),(3,3),(5,5)]
    - 3
    - 2
    - 1


In [1]:
pOptimizers = ['adam', 'adadelta', 'adagrad']
pOptimizersLr = [0.01, 0.1]
pOptimizersRho = [0.01, 0.99]
pOptimizersBeta = [0.9, 0.99]
pLosses = ['categorical_crossentropy', 'mean_squared_error', 'mean_squared_error']
pnDense = [1,2,3]
pDenseSize = [16, 32,64]
pnConv = [2,4,6]
pConvSize = [64,96,128]
pConvKernel = [(2,2),(3,3)]

In [2]:
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
import random

os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta,Adam,Adagrad

Using TensorFlow backend.


In [3]:

numModels = 30
modelsParam = [{
    'optimizer':np.random.choice(pOptimizers),
    'optimizer_lr': random.uniform(pOptimizersLr[0],pOptimizersLr[1]),
    'optimizer_rho': random.uniform(pOptimizersRho[0],pOptimizersRho[1]),
    'optimizer_beta1': random.uniform(pOptimizersBeta[0],pOptimizersBeta[1]),
    'optimizer_beta2': random.uniform(pOptimizersBeta[0],pOptimizersBeta[1]),
    'loss': np.random.choice(pLosses),
    'n_dense': np.random.choice(pnDense),
    'dense_size': np.random.choice(pDenseSize,max(pnDense)),
    'n_conv': np.random.choice(pnConv),
    'conv_size': np.random.choice(pConvSize,1),
    'conv_kernel':[pConvKernel[x] for x in  np.random.choice(len(pConvKernel),max(pnConv))]
} for i in range(numModels)]
modelsAcc = [ 0 for i in range(numModels)]

In [4]:
def buildModel(dic):
    input_shape = (32, 32, 3)
    model = Sequential()
    
    model.add(Conv2D(128, kernel_size=(5, 5), input_shape=input_shape))
    model.add(Activation('relu'))
    
    conv_size = dic['conv_size'][0]
    for i in range(dic['n_conv']):
        model.add(Conv2D(conv_size, 
                         kernel_size=dic['conv_kernel'][i], 
                         padding='same'))
        model.add(Activation('relu'))
        model.add(Conv2D(conv_size, 
                         kernel_size=dic['conv_kernel'][i], 
                         padding='same'))
        model.add(Activation('relu'))
        if conv_size>16:
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.25))
        conv_size //= 2

    model.add(Flatten())  

    dense_size = dic['dense_size'][0]
    for i in range(dic['n_dense']):
        model.add(Dense(dense_size))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        dense_size //= 2

    model.add(Dense(10))
    model.add(Activation('softmax'))

    optimizer = None
    if dic['optimizer'] == 'adam':
        optimizer = Adam(lr = dic['optimizer_lr'],
                         beta_1 = dic['optimizer_beta1'],
                         beta_2 = dic['optimizer_beta2'])
    if dic['optimizer'] == 'adadelta':
        optimizer = Adadelta(lr = dic['optimizer_lr'],
                         rho = dic['optimizer_rho'])
    if dic['optimizer'] == 'adagrad':
        optimizer = Adagrad(lr = dic['optimizer_lr'])
    model.compile(loss=dic['loss'],optimizer='adam',metrics=['accuracy'])
    return model

In [5]:
def save_labels(arr, filename):
    pd_array = pd.DataFrame(arr)
    pd_array.index.names = ["Id"]
    pd_array.columns = ["Prediction"]
    pd_array.to_csv(filename)

def load_labels(filename):
    return pd.read_csv(filename, index_col=0).values.ravel()

In [6]:
X_train = np.load("E:\\Desktop\\dane_ml\\X_train.npy")
y_train = load_labels("E:\\Desktop\\dane_ml\\y_train.csv")
X_test = np.load("E:\\Desktop\\dane_ml\\X_test.npy")

# Mój keras ma kolorki na końcu
X_train = X_train.reshape(X_train.shape[0], 3, 32, 32).transpose(0,2,3,1)
X_test = X_test.reshape(X_test.shape[0], 3, 32, 32).transpose(0,2,3,1)


In [7]:
first_X_train, first_X_test, first_y_train, first_y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=43)
first_y_train = to_categorical(first_y_train, num_classes=10)
first_y_test = to_categorical(first_y_test, num_classes=10)

In [8]:
for i,params in enumerate(modelsParam):
    try:
        print ("Testing model: ", i)
        model = buildModel(params)
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        sess = tf.Session(config=config)
        model.fit(first_X_train, first_y_train, batch_size=128, epochs=3, verbose=1, validation_data=(first_X_test, first_y_test))
        score = model.evaluate(first_X_test, first_y_test, verbose=0)[1]

        if score < 0.13:
            print ("Skipping this model")
            continue

        model = buildModel(params)
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        sess = tf.Session(config=config)
        model.fit(first_X_train, first_y_train, batch_size=25, epochs=25, verbose=1, validation_data=(first_X_test, first_y_test))
        score = model.evaluate(first_X_test, first_y_test, verbose=0) [1]
        modelsAcc[i] = score
    except:
        print("Pomijam!")
        continue

Testing model:  0
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 60s - loss: 0.0881 - acc: 0.1624 - val_loss: 0.0854 - val_acc: 0.2226
Epoch 2/3
45000/45000 [==============================] - 51s - loss: 0.0811 - acc: 0.2835 - val_loss: 0.0758 - val_acc: 0.3536
Epoch 3/3
45000/45000 [==============================] - 51s - loss: 0.0767 - acc: 0.3455 - val_loss: 0.0728 - val_acc: 0.4030
Train on 45000 samples, validate on 5000 samples
Epoch 1/25
45000/45000 [==============================] - 72s - loss: 0.0881 - acc: 0.1655 - val_loss: 0.0819 - val_acc: 0.2552
Epoch 2/25
45000/45000 [==============================] - 70s - loss: 0.0822 - acc: 0.2647 - val_loss: 0.0785 - val_acc: 0.3180
Epoch 3/25
45000/45000 [==============================] - 71s - loss: 0.0787 - acc: 0.3198 - val_loss: 0.0743 - val_acc: 0.3774
Epoch 4/25
45000/45000 [==============================] - 71s - loss: 0.0757 - acc: 0.3618 - val_loss: 0.0729 - val_acc

45000/45000 [==============================] - 27s - loss: 0.1797 - acc: 0.1014 - val_loss: 0.1801 - val_acc: 0.0996
Epoch 3/3
45000/45000 [==============================] - 27s - loss: 0.1796 - acc: 0.1003 - val_loss: 0.1801 - val_acc: 0.0996
Skipping this model
Testing model:  4
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 39s - loss: 0.1369 - acc: 0.0966 - val_loss: 0.1810 - val_acc: 0.0950
Epoch 2/3
45000/45000 [==============================] - 37s - loss: 0.1344 - acc: 0.0995 - val_loss: 0.1810 - val_acc: 0.0950
Epoch 3/3
45000/45000 [==============================] - 37s - loss: 0.1343 - acc: 0.0992 - val_loss: 0.1810 - val_acc: 0.0950
Skipping this model
Testing model:  5
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 47s - loss: 2.3294 - acc: 0.1008 - val_loss: 2.3029 - val_acc: 0.0950
Epoch 2/3
45000/45000 [==============================] - 45s - loss: 2.299

45000/45000 [==============================] - 61s - loss: 0.0900 - acc: 0.0984 - val_loss: 0.0900 - val_acc: 0.0950
Epoch 18/25
45000/45000 [==============================] - 62s - loss: 0.0900 - acc: 0.0982 - val_loss: 0.0900 - val_acc: 0.1048- ETA: 5s - loss: 0.0900  - ETA: 4s - loss: - ETA: 2s - loss - ETA: 1s - loss
Epoch 19/25
45000/45000 [==============================] - 62s - loss: 0.0900 - acc: 0.0984 - val_loss: 0.0900 - val_acc: 0.0978
Epoch 20/25
45000/45000 [==============================] - 62s - loss: 0.0900 - acc: 0.1000 - val_loss: 0.0900 - val_acc: 0.0950 0.0900 - acc: 0.100
Epoch 21/25
45000/45000 [==============================] - 62s - loss: 0.0900 - acc: 0.0952 - val_loss: 0.0900 - val_acc: 0.0950
Epoch 22/25
45000/45000 [==============================] - 62s - loss: 0.0900 - acc: 0.0997 - val_loss: 0.0900 - val_acc: 0.0994
Epoch 23/25
45000/45000 [==============================] - 62s - loss: 0.0900 - acc: 0.0976 - val_loss: 0.0900 - val_acc: 0.1000
Epoch 24/25


45000/45000 [==============================] - 45s - loss: 0.1751 - acc: 0.0988 - val_loss: 0.1790 - val_acc: 0.1048
Skipping this model
Testing model:  17
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 47s - loss: 0.1784 - acc: 0.1012 - val_loss: 0.1782 - val_acc: 0.1088
Epoch 2/3
45000/45000 [==============================] - 45s - loss: 0.1784 - acc: 0.1010 - val_loss: 0.1798 - val_acc: 0.1008
Epoch 3/3
45000/45000 [==============================] - 45s - loss: 0.1713 - acc: 0.0994 - val_loss: 0.1798 - val_acc: 0.1008
Skipping this model
Testing model:  18
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 42s - loss: 0.1488 - acc: 0.1001 - val_loss: 0.0900 - val_acc: 0.0978
Epoch 2/3
45000/45000 [==============================] - 40s - loss: 0.0900 - acc: 0.0993 - val_loss: 0.0900 - val_acc: 0.1008
Epoch 3/3
45000/45000 [==============================] - 40s - loss: 0.0

45000/45000 [==============================] - 80s - loss: 0.0656 - acc: 0.4797 - val_loss: 0.0643 - val_acc: 0.4778
Epoch 13/25
45000/45000 [==============================] - 80s - loss: 0.0648 - acc: 0.4853 - val_loss: 0.0623 - val_acc: 0.5042
Epoch 14/25
45000/45000 [==============================] - 80s - loss: 0.0646 - acc: 0.4874 - val_loss: 0.0610 - val_acc: 0.5196
Epoch 15/25
45000/45000 [==============================] - 80s - loss: 0.0633 - acc: 0.5010 - val_loss: 0.0627 - val_acc: 0.4968
Epoch 16/25
45000/45000 [==============================] - 80s - loss: 0.0632 - acc: 0.4993 - val_loss: 0.0605 - val_acc: 0.5152
Epoch 17/25
45000/45000 [==============================] - 80s - loss: 0.0616 - acc: 0.5164 - val_loss: 0.0611 - val_acc: 0.5258
Epoch 18/25
45000/45000 [==============================] - 80s - loss: 0.0611 - acc: 0.5206 - val_loss: 0.0602 - val_acc: 0.5242
Epoch 19/25
45000/45000 [==============================] - 80s - loss: 0.0603 - acc: 0.5290 - val_loss: 0.057

In [11]:
print(modelsAcc)

[0.495, 0, 0.10879999999999999, 0, 0, 0, 0.32100000000000001, 0, 0.095000000000000001, 0, 0, 0, 0, 0, 0, 0.30480000000000002, 0, 0, 0, 0.092999999999999999, 0, 0, 0, 0.59340000000000004, 0, 0, 0, 0, 0, 0]


In [13]:
#selected model 
ind = np.argmax(modelsAcc)
print(modelsAcc[ind])
print(modelsParam[ind])

0.5934
{'optimizer_rho': 0.4669651142511834, 'conv_kernel': [(3, 3), (2, 2), (2, 2), (3, 3), (2, 2), (2, 2)], 'optimizer_lr': 0.09471348656871778, 'n_dense': 1, 'dense_size': array([64, 16, 16]), 'conv_size': array([128]), 'optimizer': 'adadelta', 'optimizer_beta2': 0.9613258235798201, 'loss': 'mean_squared_error', 'n_conv': 6, 'optimizer_beta1': 0.9424973706740825}


In [17]:
y_train = to_categorical(y_train, num_classes=10)

In [18]:
#uczę wybrany model na wszytskich danych
model = buildModel(modelsParam[ind])
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
model.fit(X_train, y_train, batch_size=125, epochs=100, verbose=1, validation_data=(first_X_test, first_y_test))

Train on 50000 samples, validate on 5000 samples
Epoch 1/100
50000/50000 [==============================] - 65s - loss: 0.0888 - acc: 0.1365 - val_loss: 0.0829 - val_acc: 0.2732
Epoch 2/100
50000/50000 [==============================] - 62s - loss: 0.0814 - acc: 0.2871 - val_loss: 0.0777 - val_acc: 0.3514
Epoch 3/100
50000/50000 [==============================] - 62s - loss: 0.0773 - acc: 0.3464 - val_loss: 0.0743 - val_acc: 0.3806
Epoch 4/100
50000/50000 [==============================] - 62s - loss: 0.0743 - acc: 0.3859 - val_loss: 0.0710 - val_acc: 0.4178
Epoch 5/100
50000/50000 [==============================] - 61s - loss: 0.0719 - acc: 0.4104 - val_loss: 0.0672 - val_acc: 0.4520
Epoch 6/100
50000/50000 [==============================] - 61s - loss: 0.0696 - acc: 0.4371 - val_loss: 0.0646 - val_acc: 0.4838
Epoch 7/100
50000/50000 [==============================] - 61s - loss: 0.0678 - acc: 0.4585 - val_loss: 0.0646 - val_acc: 0.4738
Epoch 8/100
50000/50000 [=======================

50000/50000 [==============================] - 61s - loss: 0.0359 - acc: 0.7459 - val_loss: 0.0294 - val_acc: 0.7934
Epoch 64/100
50000/50000 [==============================] - 61s - loss: 0.0356 - acc: 0.7475 - val_loss: 0.0295 - val_acc: 0.7886
Epoch 65/100
50000/50000 [==============================] - 61s - loss: 0.0355 - acc: 0.7476 - val_loss: 0.0302 - val_acc: 0.7832
Epoch 66/100
50000/50000 [==============================] - 61s - loss: 0.0351 - acc: 0.7506 - val_loss: 0.0282 - val_acc: 0.8000
Epoch 67/100
50000/50000 [==============================] - 60s - loss: 0.0350 - acc: 0.7520 - val_loss: 0.0303 - val_acc: 0.7858
Epoch 68/100
50000/50000 [==============================] - 60s - loss: 0.0350 - acc: 0.7533 - val_loss: 0.0286 - val_acc: 0.7970
Epoch 69/100
50000/50000 [==============================] - 60s - loss: 0.0343 - acc: 0.7578 - val_loss: 0.0275 - val_acc: 0.8052
Epoch 70/100
50000/50000 [==============================] - 60s - loss: 0.0342 - acc: 0.7597 - val_loss

In [19]:
save_labels([np.argmax(x)for x in model.predict(X_test)], "y_pred.csv")

In [ ]:
print model.